# ECE1524 Project: Routerless IP Networking 

Yida Wang

Tianran Xu

## README

* Requirements:
  * Python 3
  * Jupyter Notebook
  * Mininet
  * ONOS
* Important files:
  * Abilene Topology and associated dataset (traffic matrices): http://totem.run.montefiore.ulg.ac.be/files/data/abilene-TM.tar.gz
    (extract under ECE1524/ to ensure the Python program can run successfully)
  * Jupyter Notebook HTML Report: ECE1524/project_notebook.html
  * Jupyter Notebook: ECE1524/project_notebook.ipynb
  * Abilene topology for Mininet: ECE1524/abilene.py

In [8]:
# setup

from Parser import *

graph = Parser.parse_topology(False)

## Setup Mininet for Abilene

To demonstrate the deployment, we are going to design flow rules based on the Valiant routing strategy are used for flow rules. ONOS is selected as the controller. Abilene topology for Mininet can be found in ECE1524/abilene.py.

ONOS Core REST API is used for the interaction with the controller. We can use `GET /topology` to get overview of current topology.

In [9]:
import requests
import json

r = requests.get('http://127.0.0.1:8181/onos/v1/topology/', auth=('karaf', 'karaf'))
r.json()

{'clusters': 1, 'devices': 12, 'links': 30, 'time': 9741776547998}

## Deploy flow rules

`POST /flows/{deviceId}` is used to creates and install a new flow rule for the specified device. 

In [5]:
# setup
node_mapping = [
    "of:0000000000000001", # ATLA-M5
    "of:0000000000000002", # ATLAng
    "of:0000000000000003", # CHINng
    "of:0000000000000004", # DNVRng
    "of:0000000000000005", # HSTNng
    "of:0000000000000006", # IPLSng
    "of:0000000000000007", # KSCYng
    "of:0000000000000008", # LOSAng
    "of:0000000000000009", # NYCMng
    "of:000000000000000a", # SNVAng
    "of:000000000000000b", # STTLng
    "of:000000000000000c"  # WASHng
]

mac_address_mapping = [
    ["00:00:00:00:00:01", "C0:01:2B:D4:00:00"], # ATLA-M5
    ["00:00:00:00:00:02", "C0:02:26:4C:00:00"], # ATLAng
    ["00:00:00:00:00:03", "C0:03:30:48:00:00"], # CHINng
    ["00:00:00:00:00:04", "C0:04:0A:18:00:00"], # DNVRng
    ["00:00:00:00:00:05", "C0:05:38:D0:00:00"], # HSTNng
    ["00:00:00:00:00:06", "C0:06:2E:2C:00:00"], # IPLSng
    ["00:00:00:00:00:07", "C0:07:08:28:00:00"], # KSCYng
    ["00:00:00:00:00:08"], # LOSAng
    ["00:00:00:00:00:09"], # NYCMng
    ["00:00:00:00:00:0a"], # SNVAng
    ["00:00:00:00:00:0b"], # STTLng
    ["00:00:00:00:00:0c"]  # WASHng
]

node_used_port = [1] * len(graph.link_list)
link_tmp_dict = {}

link_mapping = []

for i in range(0, len(graph.link_list)):
    link_mapping.append(
        {"src": {"port": "", "device": ""}, 
         "dst": {"port": "", "device": ""}})
    link_from = graph.node_list.index(graph.link_list[i].link_from)
    link_mapping[i]["src"]["device"] = node_mapping[link_from]
    
    link_to = graph.node_list.index(graph.link_list[i].link_to)
    link_mapping[i]["dst"]["device"] = node_mapping[link_to]
    
    if (link_to, link_from) not in link_tmp_dict:
        # assign port to node based on the ONOS mechanism
        node_used_port[link_from] += 1
        node_used_port[link_to] += 1
        
        link_mapping[i]["src"]["port"] = str(node_used_port[link_from])
        link_mapping[i]["dst"]["port"] = str(node_used_port[link_to])
        link_tmp_dict[(link_from, link_to)] = (str(node_used_port[link_from]), str(node_used_port[link_to]))
    else:
        link_mapping[i]["src"]["port"] = link_tmp_dict[(link_to, link_from)][1]
        link_mapping[i]["dst"]["port"] = link_tmp_dict[(link_to, link_from)][0]

# install flow rules
for i in range(0, len(graph.node_list)):
    src = graph.node_list[i]
    # Valiant routing strategy
    metric, prev = graph.shortest_path_valiant(src)
    for j in range(0, len(graph.node_list)):
        if i != j:
            path = []
            hop = graph.node_list[j]
            while hop != src:
                link_index = graph.link_list.index(graph.find_link(prev[hop], hop))
                path.append(link_index)
                hop = prev[hop]
            for l in range(0, len(path)):
                if l == len(path) - 1:
                    # src
                    for x in mac_address_mapping[i]:
                        for y in mac_address_mapping[j]:
                            if mac_address_mapping[i].index(x) == 0:
                                port = 1
                            else:
                                # GNS3 router
                                port = node_used_port[i] + 1
                            flow = {"isPermanent": "true",
                                 "priority": 50000,
                                 "timeout": 0,
                                 "selector": 
                                     {"criteria": 
                                        [{"type": "IN_PORT", "port": port}, 
                                         {"type": "ETH_DST", "mac": y}, 
                                         {"type": "ETH_SRC", "mac": x}]},
                                 "treatment":
                                     {"instructions": [{"type": "OUTPUT", "port":
                                                        link_mapping[path[l]]["src"]["port"]}]}}
                            requests.post('http://127.0.0.1:8181/onos/v1/flows/' +\
                                          link_mapping[path[l]]["src"]["device"], 
                                          data=json.dumps(flow), auth=('karaf', 'karaf'))
                    
                if l == 0:
                    # dst
                    for x in mac_address_mapping[i]:
                        for y in mac_address_mapping[j]:
                            if mac_address_mapping[j].index(y) == 0:
                                port = 1
                            else:
                                # GNS3 router
                                port = node_used_port[j] + 1
                            flow = {"isPermanent": "true",
                                 "priority": 50000,
                                 "timeout": 0,
                                 "selector": 
                                     {"criteria": 
                                        [{"type": "IN_PORT", "port": link_mapping[path[l]]["dst"]["port"]}, 
                                         {"type": "ETH_DST", "mac": y}, 
                                         {"type": "ETH_SRC", "mac": x}]},
                                 "treatment":
                                     {"instructions": [{"type": "OUTPUT", "port": port}]}}
                            requests.post('http://127.0.0.1:8181/onos/v1/flows/' +\
                                          link_mapping[path[l]]["dst"]["device"], 
                                          data=json.dumps(flow), auth=('karaf', 'karaf')) 
                            
                else:
                    for x in mac_address_mapping[i]:
                        for y in mac_address_mapping[j]:
                            flow = {"isPermanent": "true",
                                 "priority": 50000,
                                 "timeout": 0,
                                 "selector": 
                                     {"criteria": 
                                        [{"type": "IN_PORT", "port": link_mapping[path[l]]["dst"]["port"]}, 
                                         {"type": "ETH_DST", "mac": y}, 
                                         {"type": "ETH_SRC", "mac": x}]},
                                 "treatment":
                                     {"instructions": [{"type": "OUTPUT",
                                                        "port": link_mapping[path[l - 1]]["src"]["port"]}]}}
                            requests.post('http://127.0.0.1:8181/onos/v1/flows/' + \
                                          link_mapping[path[l - 1]]["src"]["device"], 
                                          data=json.dumps(flow), auth=('karaf', 'karaf'))


We can query all flow rules in the system using `GET /flows` to ensure all flows are correctly installed. We can also observe that 'packets' and 'bytes' from the response keep incrementing as we send traffic between the nodes, which confirms the flow rules are actually used.

In [ ]:
r = requests.get('http://127.0.0.1:8181/onos/v1/flows/', auth=('karaf', 'karaf'))
r.json()